# Introduction
Let's say you want to do a curve collapse analsis of an observable $O$ by fitting data to one of the following fit ansatz:
$$O(K,N_{\mathrm{s}})=N_{\mathrm{s}}^{\gamma_{O}}\mathcal{F}_{O}\Big((K/K_{\mathrm{c}}-1)N_{\mathrm{s}}^{1/\nu}\Big)$$
for 2nd-order or
$$O(K,N_{\mathrm{s}})=N_{\mathrm{s}}^{\gamma_{O}}\mathcal{F}_{O}\Big(N_{\mathrm{s}}\exp\big(\mbox{-}\zeta |K/K_{\mathrm{c}}-1|^{-\nu}\big)\Big)$$
for $\infty$-order (BKT). This notebook will guide you through how do to this analysis. To see how to scale this up to a full empirical Bayes analysis, see `examples/potts2_example.ipynb`, `examples/potts3_example.ipynb`, `examples/clock4_example.ipynb` or `examples/clockinf_example.ipynb`. 

# Import modules
We need to load up some modules. First, let's import GVar and Numpy.

In [1]:
import gvar as gv # Peter Lepage's GVar module
import numpy as np # NumPy for number crunching

Next, we import all of SwissFit's modules.

In [2]:
from swissfit import fit # SwissFit fitter module
from swissfit.optimizers import scipy_basin_hopping # Basin hopping global optimizer
from swissfit.optimizers import scipy_least_squares # Trust region reflective local optimizer
from swissfit.machine_learning import radial_basis # Module for radial basis function network

# Grab your data
For the curve collapse analysis, our input data is a list of pairs $(K,N_{\mathrm{s}})$ and our output data is a list of measurements $O(K,N_{\mathrm{s}})$. You need to write your own code to grab the data that you want to do your analysis with. `SwissFit` will take this data in as a dictionary that is organized as 
```
data = {'x': [[K_1, N_2], [K_2, N_2], ...], 'y': [O_1, O_2, ...]}
```
where `data['x']=[[K_1, N_2], [K_2, N_2], ...]` is a list of $(K,N_{\mathrm{s}})$ pairs and `data['y']=[O_1, O_2, ...]` is a list of `GVar` variables for $O(K,N_{\mathrm{s}})$. Here's a skeleton of how you might do this.

In [ ]:
"""
Insert your code that grabs your list of (K,N) pairs [[K_1, N_2], [K_2, N_2], ...] 
and output GVar variables [O_1, O_2, ...] here. Below, your input pairs are saved into
"input" and your output variables are saved into "output" as an example.
"""

# Input data
input = # [[K_1, N_1], [K_2, N_2], ...]

# Output data
output = # [O_1, O_2, ....]

# Save your data as a dictionary (see above for description)
data = {'x': input, 'y': output}

# Set up radial basis function network
Now we need to create our radial basis function network (RBFN). This proceeds in two steps. First, we create a dictionary that defines the topology of the RBFN. The following network will have a single hidden layer with two nodes in that layer.

In [ ]:
# Specify the number of hidden nodes
hidden_nodes = 2 # *** Change if you want more hidden nodes ***

# Specify network topology
network_topology = {
    'lyr1': { # Hidden layer
        'in': 1, 'out': hidden_nodes, 
        'activation': 'exp', # Exponential activation
    },
    'lyr2': { # Output layer
        'in': hidden_nodes, 'out': 1,
        'activation': 'linear' # Linear activation
    }
}

Now we create the RBFN by simply passing `network_topology` through the RBFN constructor.

In [ ]:
# Create radial basis function network
neural_network = radial_basis.RadialBasisNeuralNetwork(network_topology)

# Define your fit function
Now that we have the radial basis function network set up, let's define the fit function. For 2nd-order scaling, 
$$\mathrm{fit\_function}(K,N_{\mathrm{s}})=N_{\mathrm{s}}^{\gamma_{O}}\mathcal{F}_{O}\Big((K/K_{\mathrm{c}}-1)N_{\mathrm{s}}^{1/\nu}\Big),$$
this could be

In [ ]:
# 2nd-order scaling
def fit_fcn_2nd_order(b, l, p):
    # This is the scaling function parameterized by the neural network
    F_O = np.ravel(neural_network.out((b * p['c'][0] - 1.) * l**p['c'][1], p))

    # This is N_s**gamma_O * F_O
    return F_O * l**p['c'][-1]

If the operator that you are measuring has $\gamma_{O}=0$, just don't multiply `F_O` by `l**p['c'][-1]`. In the above example, you'll see that the critical parameters are specified in a dictionary `p` with 
```
p['c'] = [<1/K_c>, <1/nu>, <gamma_O>].
```
The parameters of the RBFN are contained in other dictionary entires of `p`.

Infinite-order scaling, 
$$\mathrm{fit\_function}(K,N_{\mathrm{s}})=N_{\mathrm{s}}^{\gamma_{O}}\mathcal{F}_{O}\Big(N_{\mathrm{s}}\exp\big(\mbox{-}\zeta |K/K_{\mathrm{c}}-1|^{-\nu}\big)\Big),$$
is specified in a similar way as follows.

In [ ]:
# infinite-order (BKT) scaling
def fit_fcn_infinite_order(b, l, p):
    # This is the scaling function parameterized by the neural network
    F_O = np.ravel(neural_network.out(
        l / gv.exp(p['c'][2] * gv.abs(b * p['c'][0] - 1.)**(-p['c'][1])), # Argument of F_O
        p # Parameters of the RBFN
    ))

    # This is N_s**gamma_O * F_O
    return F_O * l**p['c'][-1]

Again, you'll notice that the critical parameters are contained in a dictionary `p`, with
```
p['c'] = [<1/K_c>, <nu>, <zeta>, <gamma_O>]
```
and the RBFN parameters contained as other entries of the `p` dictionary.

# Set priors and starting values for parameters
Now that we have our fit function, we need to get our priors. You can do this any way that you want, as long as everything is in a dictionary that `SwissFit` accepts. Take the code below as a template. You can also just not specify any priors if you don't feel like it. It's up to you.

In [ ]:
# Initialize your dictionary of priors
prior = {}

# Specify priors
prior['c'] = # [fill with GVar variables corresponding to the priors for the critical paramters]

If you also want to specify ridge regression priors for the RBFN, the code is as follows. Note that *this is optional*. You don't need to specify a ridge regression prior for the network, but it can (and often does) help prevent overfitting.

In [ ]:
# Mean/width of ridge regression prior
ridge_mean = 0. # Mean
ridge_width = 100. # Width - tune to a smaller value if you are overfitting

# Specify ridge regression prior.
prior = neural_network.network_priors(
    prior_choice_weight = { # Prior for weights
        'lyr2': { # Only for output layer
            'prior_type': 'ridge_regression', # Type of prior
            'mean': ridge_mean, # Mean of zero
            'standard_deviation': ridge_width # Width of lambda
        }
    }, 
    prior = prior # Take in already-specified prior dictionary and modify it
)

Your fit also needs a starting point. Below is a skeleton of how to do that.

In [ ]:
# Initialize dictionary of starting values
p0 = {}

# Initialize critical parameters (if you already specified these as priors, you don't have to do this)
p0['c'] = # [fill with starting values for critical parameters]

# Initialize the parameters of the radial basis function network
p0 = neural_network.initialize_parameters(initialization = 'zero', p0 = p0)

# Set up fitter
Now we will create the `SwissFit` fitter `fitter`. The `fitter` is `SwissFit`'s analogue of `Lsqfit`'s `nonlinear_fit`. The `fitter` will optionally take in a function called `prior_transformation_function` that takes the priors and transforms them into log priors, which forces whatever parameters that you want to be positivite. 

In [ ]:
# Define a function that transforms the priors into log priors to force positivity on critical parameters *** OPTIONAL ***
log_priors = {'c': lambda x: gv.log(x)} # *** OPTIONAL ***

# Create SwissFit fitter
fitter = fit.SwissFit(
    udata = data, # Fit data; "data = data" is also acceptable - "udata" means "uncorrelated"
    uprior = prior, # Priors; "prior = prior" is also acceptable - "uprior" means "uncorrelated"
    p0 = p0, # Starting values for parameters
    fit_fcn = fit_fcn, # Fit function
    prior_transformation_fcn = log_priors # Transformation of prior "c" to "log(c)" *** OPTIONAL ***
)

# Set up the optimization algorithms
Now we need to set up the optimization algorithms. First, let's create an object that represents the trust region reflective algorithm:

In [ ]:
# Create trust region reflective local optimizer from SciPy - fitter will save reference to local_optimizer for basin hopping
local_optimizer = scipy_least_squares.SciPyLeastSquares(fitter = fitter)

Now that we've set up the local optimization algorithm, we can create our basin hopping global optimization algorithm as follows:

In [ ]:
# Basin hopping parameters
niter_success = 100 # Number of iterations with same best fit parameters for basin hopping to "converge"
niter = 10000 # Upper bound on total number of basin hopping iterations
T = 1. # Temperature hyperparameter for basin hopping

# Basin hopping global optimizer object instantiation
global_optimizer = scipy_basin_hopping.BasinHopping(
    fitter = fitter, # Fit function is the "calculate_residual" method of fitter object
    optimizer_arguments = {
        'niter_success': niter_success,
        'niter': niter,
        'T': T,
    }
)

# Do fit
Now that everything is set up, doing the fit is a single line of code

In [ ]:
fitter(global_optimizer)

If you want to print out the result of the fit:

In [ ]:
print(fitter)

If you want to grab your (correlated) fit parameters:

In [ ]:
fit_parameters = fitter.p

If you want to get the value of your fit at a specific $(K,N_{\mathrm{s}})$:
```
value = your_fit_function(K, Ns, fit_parameters)
```
where `your_fit_function` is either `fit_fcn_2nd_order` or `fit_fcn_infinite_order`.